In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import ipdb
from bs4 import BeautifulSoup

# Inputs

In [2]:
df = pd.read_csv('../raw_data/all_clean.csv')

In [4]:
df = df[['Id', 'Player']].drop_duplicates()
df.reset_index(inplace=True)
df.drop(columns="index", inplace=True)

In [5]:
df

,Id,Player
0,1,Greg Oden
1,2,Carl Landry
2,3,Gabe Pruitt
3,4,Glen Davis
4,5,Jermareo Davidson
...,...,...
63964,172607,Will Mcaloney
63965,172628,Kent Salado
63966,172801,Prince Rivero
63967,175140,Adrian Riesgo


# Soup

In [6]:
pid = 1
url = f'https://basketball.realgm.com/player/a/Summary/{pid}'

In [7]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [8]:
box = soup.find(class_="profile-box")
box

<div class="profile-box">
<div class="wrapper clearfix container">
<div class="half-column-left" style="margin: 0;">
<h2 style="margin-top: 0;">Greg Oden <span class="feature">C</span>  <span class="feature">#20</span></h2>
<img src="/images/nba/4.2/profiles/photos/2006/Oden_Greg_mia_1314.jpg" style="border: 1px solid #000; float: left; margin-right: 15px; margin-top:5px;"/>
<p><strong>Born:</strong> <a href="/info/birthdays/19880122/1">Jan 22, 1988</a> (32 years old)</p>
<p><strong>Birthplace/Hometown:</strong> <a href="/info/birth_cities/188/Buffalo-NY-United-States">Buffalo, New York</a></p>
<p><strong>Nationality:</strong> <a href="/info/nationality/1/United-States/O">United States</a></p>
<p><strong>Height:</strong> 7-0 (213cm)     <strong>Weight:</strong> 273 (124kg)</p>
<p><strong>Website:</strong> <a href="http://www.gregoden52.com/" target="_blank">http://www.gregoden52.com/</a></p>
</div>
<div class="half-column-right" style="margin: 0;">
<img src="/images/basketball/5.0/team

# Info

In [9]:
position = box.select('h2 > span')[0].text
position

'C'

In [10]:
photo = box.find('img', src=True)['src']
photo

'/images/nba/4.2/profiles/photos/2006/Oden_Greg_mia_1314.jpg'

In [11]:
birthdate = box.find("strong", string="Born:").parent.find('a').text
birthdate

'Jan 22, 1988'

In [12]:
birthplace = box.find("strong", string="Birthplace/Hometown:").parent.find('a').text
birthplace

'Buffalo, New York'

In [13]:
nationality = box.find("strong", string="Nationality:").parent.find('a').text
nationality

'United States'

In [14]:
size = box.find("strong", string="Height:").parent.text.split()
height = size[1]
weight = size[4]
height, weight

('7-0', '273')

In [15]:
nba_status = box.find("strong", string="Current NBA Status:").parent.text.split(":")[1].strip()
nba_status

'Unrestricted Free Agent'

In [16]:
agent = box.find("strong", string="Agent:").parent.find('a').text
agent

'Bill Duffy'

In [17]:
hs_info = box.find("strong", string="High School:").parent
highschool = hs_info.find('a').text
hslocation = hs_info.text.split('[')[1].strip(']')
highschool, hslocation

('Lawrence North High School', 'Indianapolis, Indiana')

In [18]:
draft_entry = box.find("strong", string="Draft Entry:").parent.find('a').text
draft_entry

'2007 NBA Draft'

In [19]:
draft_early = box.find("strong", string="Early Entry Info:").parent.find('a').text
draft_early

'2007 Early Entrant'

In [20]:
drafted = box.find("strong", string="Drafted:").parent.find('a').text
drafted

'Round 1, Pick 1, Portland Trail Blazers'

In [21]:
predraft_team = box.find("strong", string="Pre-Draft Team:").parent.find('a').text
predraft_team

'Ohio State'

# Basic Function

In [22]:
def scrape_player(id):
    url = f'https://basketball.realgm.com/player/a/Summary/{id}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    box = soup.find(class_="profile-box")
    
    if box:
        position = box.select('h2 > span')[0].text
        
        birthdate_info = box.find("strong", string="Born:")
        if birthdate_info: 
            birthdate = birthdate_info.parent.find('a').text
        else:
            birthdate = None
        
        birthplace_info = box.find("strong", string="Birthplace/Hometown:")
        if birthplace_info: 
            birthplace = birthplace_info.parent.find('a').text
        else:
            birthplace = None
        
        size_info = box.find("strong", string="Height:")
        if size_info:
            size = size_info.parent.text.split()
            height, weight = size[1], size[4]
        else:
            height, weight = None, None
            
        nba_status_info = box.find("strong", string="Current NBA Status:")
        if nba_status_info: 
            nba_status = nba_status_info.parent.text.split(":")[1].strip()
        else:
            nba_status = None
            
        agent_info = box.find("strong", string="Agent:")
        if agent_info: 
            agent = agent_info.parent.find('a').text
        else:
            agent = None
        
        hs_info = box.find("strong", string="High School:")
        if hs_info:
            hs = hs_info.parent
            highschool = hs.find('a').text
            hslocation = hs.text.split('[')[1].strip(']')
        else:
            highschool = None
            hslocation = None
            
        draft_entry_info = box.find("strong", string="Draft Entry:")
        if draft_entry_info: 
            draft_entry = draft_entry_info.parent.text
        else:
            draft_entry = None
            
        draft_early_info = box.find("strong", string="Early Entry Info:")
        if draft_early_info: 
            draft_early = draft_early_info.parent.find('a').text
        else:
            draft_early = None
            
        drafted_info = box.find("strong", string="Drafted:")
        if drafted_info: 
            drafted = drafted_info.parent.text
        else:
            drafted = None
            
        predraft_team_info = box.find("strong", string="Pre-Draft Team:")
        if predraft_team_info: 
            predraft_team = predraft_team_info.parent.find('a').text
        else:
            predraft_team = None
        
        photo = box.find('img', src=True)['src']
        
        data = {'Position': position,
                'Birthdate': birthdate,
                'Birthplace': birthplace,
                'Height': height,
                'Weight': weight,
                'Status': nba_status,
                'Agent': agent,
                'Highschool': highschool,
                'HsLocation': hslocation,
                'DraftEntry': draft_entry,
                'DraftEarly': draft_early,
                'Drafted': drafted,
                'PreDraftTeam': predraft_team,
                'Photo': photo}
        return data
    
    return None

In [23]:
pid = df['Id'].sample().values[0]
print(pid)
data = scrape_player(int(pid))

9984


In [24]:
data

{'Position': 'PG',
 'Birthdate': 'Apr 3, 1984',
 'Birthplace': 'Grapevine, Texas',
 'Height': '5-10',
 'Weight': '175',
 'Status': 'Unrestricted Free Agent',
 'Agent': None,
 'Highschool': 'Colleyville Heritage High School',
 'HsLocation': 'Colleyville, Texas',
 'DraftEntry': 'Draft Entry: 2006 NBA Draft',
 'DraftEarly': None,
 'Drafted': 'Drafted: Undrafted',
 'PreDraftTeam': 'Texas Christian',
 'Photo': '/images/nba/4.2/profiles/photos/2006/player_photo.jpg'}

In [29]:
round(100/df.shape[0] * 100, 2)

0.16

# Results

In [27]:
df = pd.read_csv('../raw_data/players.csv', low_memory=False)

In [28]:
df.shape

(63969, 16)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63969 entries, 0 to 63968
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            63969 non-null  int64 
 1   Player        63969 non-null  object
 2   Position      62563 non-null  object
 3   Birthdate     52395 non-null  object
 4   Birthplace    48438 non-null  object
 5   Height        61473 non-null  object
 6   Weight        48121 non-null  object
 7   Status        63968 non-null  object
 8   Agent         8353 non-null   object
 9   Highschool    31569 non-null  object
 10  HsLocation    31569 non-null  object
 11  DraftEntry    63959 non-null  object
 12  DraftEarly    2135 non-null   object
 13  Drafted       53847 non-null  object
 14  PreDraftTeam  45390 non-null  object
 15  Photo         63969 non-null  object
dtypes: int64(1), object(15)
memory usage: 7.8+ MB


In [33]:
df

,Id,Player,Position,Birthdate,Birthplace,Height,Weight,Status,Agent,Highschool,HsLocation,DraftEntry,DraftEarly,Drafted,PreDraftTeam,Photo
0,1,Greg Oden,C,"Jan 22, 1988","Buffalo, New York",7-0,273.0,Unrestricted Free Agent,Bill Duffy,Lawrence North High School,"Indianapolis, Indiana",Draft Entry: 2007 NBA Draft,2007 Early Entrant,"Drafted: Round 1, Pick 1, Portland Trail Blazers",Ohio State,/images/nba/4.2/profiles/photos/2006/Oden_Greg...
1,2,Carl Landry,F,"Sep 19, 1983","Milwaukee, Wisconsin",6-9,248.0,Unrestricted Free Agent,Mark Bartelstein,Vincent High School,"Milwaukee, Wisconsin",Draft Entry: 2007 NBA Draft,NaN,"Drafted: Round 2, Pick 1, Seattle SuperSonics",Purdue,/images/nba/4.2/profiles/photos/2006/Landry_Ca...
2,3,Gabe Pruitt,G,"Apr 19, 1986","Los Angeles, California",6-4,185.0,Unrestricted Free Agent,NaN,Westchester High School,"Los Angeles, California",Draft Entry: 2007 NBA Draft,2007 Early Entrant,"Drafted: Round 2, Pick 2, Boston Celtics",USC,/images/nba/4.2/profiles/photos/2006/Pruitt_Ga...
3,4,Glen Davis,PF,"Jan 1, 1986","Baton Rouge, Louisiana",6-9,289.0,Unrestricted Free Agent,Zac Benalloul,Louisiana State University Laboratory School,"Baton Rouge, Louisiana",Draft Entry: 2007 NBA Draft,2007 Early Entrant,"Drafted: Round 2, Pick 5, Seattle SuperSonics",LSU,/images/nba/4.2/profiles/photos/2006/Davis_Gle...
4,5,Jermareo Davidson,PF,"Nov 15, 1984","Atlanta, Georgia",6-10,230.0,Unrestricted Free Agent,Luciano Capicchioni,Joseph Wheeler High School,"Marietta, Georgia",Draft Entry: 2007 NBA Draft,2005 Early Entrant,"Drafted: Round 2, Pick 6, Golden State Warriors",Alabama,/images/nba/4.2/profiles/photos/2006/Davidson_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63964,172607,Will Mcaloney,NaN,NaN,NaN,NaN,NaN,Draft Eligible in 2021,NaN,NaN,NaN,Draft Entry: 2021 NBA Draft,NaN,NaN,NaN,/images/nba/4.2/profiles/photos/2006/player_ph...
63965,172628,Kent Salado,NaN,NaN,NaN,NaN,NaN,Draft Eligible in 2021,NaN,NaN,NaN,Draft Entry: 2021 NBA Draft,NaN,NaN,NaN,/images/nba/4.2/profiles/photos/2006/player_ph...
63966,172801,Prince Rivero,NaN,NaN,NaN,NaN,NaN,Draft Eligible in 2021,NaN,NaN,NaN,Draft Entry: 2021 NBA Draft,NaN,NaN,NaN,/images/nba/4.2/profiles/photos/2006/player_ph...
63967,175140,Adrian Riesgo,C,"Apr 9, 2000","Oviedo, Spain",6-9,NaN,Draft Eligible in 2022,NaN,NaN,NaN,Draft Entry: 2022 NBA Draft,NaN,NaN,NaN,/images/nba/4.2/profiles/photos/2006/player_ph...


In [35]:
df['PreDraftTeam'] = df['PreDraftTeam'].map(lambda x: x.replace('Pre-Draft Team: ', '') \
                                                           .replace(' (Jr)', '') \
                                                           .replace(' (Fr)', '') \
                                                           .replace(' (Sr)', '') \
                                                          if type(x) != float else x)

In [37]:
df['Drafted'] = df['Drafted'].map(lambda x: x.replace('Drafted: ', '') if type(x) != float else x)

In [38]:
df['DraftEntry'] = df['DraftEntry'].map(lambda x: x.replace('Draft Entry: ', '') if type(x) != float else x)

In [41]:
df.sample(20)

,Id,Player,Position,Birthdate,Birthplace,Height,Weight,Status,Agent,Highschool,HsLocation,DraftEntry,DraftEarly,Drafted,PreDraftTeam,Photo
22993,31655,Tyler Hunt,F,"Jan 28, 1993","Oswego, New York",6-7,245,Unrestricted Free Agent,NaN,IMG Academy,"Bradenton, Florida",2016 NBA Draft,NaN,Undrafted,Merrimack College,/images/nba/4.2/profiles/photos/2006/Hunt_Tyle...
1865,2523,Aaron Spears,FC,"Feb 2, 1984","Chicago, Illinois",6-10,256.0,Unrestricted Free Agent,Hend Abufarie,Dunbar High School,"Dayton, Ohio",2007 NBA Draft,NaN,Undrafted,St. John's,/images/nba/4.2/profiles/photos/2006/player_ph...
7377,10442,Jio Fontan,PG,"Jan 4, 1990","Paterson, New Jersey",6-0,175,Unrestricted Free Agent,NaN,St. Anthony High School,"Jersey City, New Jersey",2013 NBA Draft,NaN,Undrafted,USC,/images/nba/4.2/profiles/photos/2006/Fontan_Ji...
12321,17374,Trent Blakely,F,NaN,"Alta Loma, California",6-4,220,Unrestricted Free Agent,NaN,Alta Loma High School,"Rancho Cucamonga, California",2010 NBA Draft,NaN,Undrafted,Cal State Bakersfield,/images/nba/4.2/profiles/photos/2006/player_ph...
50653,110980,Mykhaylo Sharko,F,"Sep 6, 1995",NaN,6-8,212,Unrestricted Free Agent,NaN,NaN,NaN,2017 NBA Draft,NaN,Undrafted,BC Kharkiv (),/images/nba/4.2/profiles/photos/2006/player_ph...
20860,29306,Vafessa Fofana,SF,"Jun 12, 1992","Paris, France",6-6,202,Unrestricted Free Agent,NaN,NaN,NaN,2014 NBA Draft,NaN,Undrafted,Denain ASC Voltaire (France),/images/nba/4.2/profiles/photos/2006/player_ph...
41454,81432,Bryce Washington,F,"Jan 25, 1996","New Orleans, Louisiana",6-6,245,Unrestricted Free Agent,Jeff Whitney,St. Augustine High School,"New Orleans, Louisiana",2018 NBA Draft,NaN,Undrafted,Louisiana,/images/nba/4.2/profiles/photos/2006/Washingto...
45675,94858,Harry Morrice,C,"Apr 13, 2000","Dundee, Scotland",6-11,235,Draft Eligible in 2023,NaN,NaN,NaN,2023 NBA Draft,NaN,NaN,North Carolina A&T,/images/nba/4.2/profiles/photos/2006/Morrice_H...
21035,29526,Mislav Peric,SG,"Aug 7, 1988",NaN,6-3,NaN,Unrestricted Free Agent,NaN,NaN,NaN,2010 NBA Draft,NaN,Undrafted,NaN,/images/nba/4.2/profiles/photos/2006/player_ph...
45905,96985,Aymar-Kabrel Boundou,PG,"Nov 11, 1982",NaN,6-1,NaN,Unrestricted Free Agent,NaN,NaN,NaN,2004 NBA Draft,NaN,Undrafted,NaN,/images/nba/4.2/profiles/photos/2006/player_ph...
